In [180]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [218]:
links_scientist = ['https://career.habr.com/vacancies?s[]=44&type=all']

links_analytic = ['https://career.habr.com/vacancies?page=1&s[]=43&type=all',
                  'https://career.habr.com/vacancies?page=2&s[]=43&type=all',
                  'https://career.habr.com/vacancies?page=3&s[]=43&type=all']

In [182]:
configurations = {
    "headers": {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"},
    "encoding": "utf-8",
    "timeout": 5,
    "should_verify_certificate": True,
    "headless_mode": True
}

In [226]:
class ParsedHabr:
  def __init__(self, link, configurations):
    self.links = link
    self.configurations = configurations
    self.data = {'date': [],
                 'company_name': [],
                 'specialization': [],
                 'vacancy': [],
                 'meta_information': [],
                 'level': [],
                 'required_skills': [],
                 'link_to_vacancy': []
                }

    self.levels = ['Младший (Junior)',
                   'Средний (Middle)',
                   'Старший (Senior)',
                   'Ведущий (Lead)']

  def parse_vacancies_page(self, soup):
    """
    Parsing vacancies from the link
    """
    all_section = soup.find_all("div", class_="vacancy-card")
    for element in all_section:
      date = element.find('div', class_="vacancy-card__date").get_text() # добавить дату с годом прям
      self.data['date'].append(date)

      company_name = element.find('div', class_="vacancy-card__company-title").get_text()
      self.data['company_name'].append(company_name)

      vacancy = element.find('div', class_="vacancy-card__title").find('a').get_text()
      self.data['vacancy'].append(vacancy)

      meta = self.get_meta(element)
      self.data['meta_information'].append(meta)

      specialization, level, skills = self.get_skills(element)
      self.data['specialization'].append(specialization)
      self.data['level'].append(level)
      self.data['required_skills'].append(skills)
      self.data['link_to_vacancy'].append(self.get_vacancy_link(element))

  @staticmethod
  def get_meta(element):
    """
    Get meta information,
    such as city, remote possibility etc
    """
    meta_info = element.find('div', class_="vacancy-card__meta").find_all('span')
    mets = []

    for el in meta_info:
      meta = el.get_text()
      if meta != ' • ':
        mets.append(meta)

    return ', '.join(mets)

  def get_skills(self, element):
    """
    Get specialization, level and required_skills
    """
    skills_list = element.find('div', class_="vacancy-card__skills").find_all('span')
    skill = []
    for el in skills_list:
      one_skill = el.get_text()
      if one_skill != ' • ':
        skill.append(one_skill)

    specialization = skill.pop(0)

    if skill[0] in self.levels:
      level = skill.pop(0)
    else:
      level = np.NaN

    return specialization, level, ', '.join(skill)

  @staticmethod
  def get_vacancy_link(element):
    """
    Get full link to vacancy
    """
    to_vacancies = 'https://career.habr.com'
    link_vac = element.find('div', class_="vacancy-card__title").find('a').get('href')

    return ''.join([to_vacancies, link_vac])

  def parse_pages(self):
    for link in self.links:
      response = requests.get(link,
                              self.configurations.get('headers'),
                              timeout=self.configurations.get('timeout'),
                              verify=self.configurations.get('should_verify_certificate'))
      soup = BeautifulSoup(response.text, 'lxml')
      self.parse_vacancies_page(soup)

  def get_dataframe(self):
    """
    Getting dataframe with all data
    """
    self.parse_pages()
    df = pd.DataFrame(self.data)
    return df

In [227]:
parsed_scientists = ParsedHabr(links_scientist,
                               configurations)
scientists = parsed_scientists.get_dataframe()

In [228]:
parsed_analytics = ParsedHabr(links_analytic,
                               configurations)
analytics = parsed_analytics.get_dataframe()

In [229]:
analytics

,date,company_name,specialization,vacancy,meta_information,level,required_skills,link_to_vacancy
0,1 апреля,Альфа-Банк,Аналитик по данным,Риск - аналитик SQL,"Москва, Полный рабочий день, Можно удаленно",Средний (Middle),"SQL, PostgreSQL, Базы данных, Apache Hadoop",https://career.habr.com/vacancies/1000138675
1,1 апреля,ДИТ г. Москвы,Аналитик по данным,Аналитик,"Москва, Полный рабочий день",Средний (Middle),"BI, BPMN, UML",https://career.habr.com/vacancies/1000135908
2,1 апреля,Почта Банк,Аналитик по данным,Главный аналитик (PL/MS SQL/Python),"Москва, Полный рабочий день",NaN,"SQL, Python",https://career.habr.com/vacancies/1000140471
3,1 апреля,beeline,Аналитик по данным,Data Analyst,"Москва, Полный рабочий день",Средний (Middle),"Python, SQL, Базы данных",https://career.habr.com/vacancies/1000133647
4,1 апреля,Холдинг Т1,Аналитик по данным,Data аналитик (Виртуальные ассистенты),"Москва, Полный рабочий день, Можно удаленно",Средний (Middle),"SQL, Python, Алгоритмы и структуры данных, Git...",https://career.habr.com/vacancies/1000132347
5,1 апреля,beeline,Аналитик по данным,Data Analyst,"Полный рабочий день, Можно удаленно",Средний (Middle),"SQL, Microsoft Excel, Python",https://career.habr.com/vacancies/1000133626
6,31 марта,YADRO,Аналитик по данным,Algorithms and Models Engineer,"Москва, Санкт-Петербург, Нижний Новгород, Полн...",Старший (Senior),"Алгоритмы, Телекоммуникации, C++, Matlab, Python",https://career.habr.com/vacancies/1000136559
7,31 марта,Холдинг Т1,Аналитик по данным,Data аналитик,"Москва, Полный рабочий день, Можно удаленно",Средний (Middle),"SQL, Python, Apache Hadoop",https://career.habr.com/vacancies/1000129571
8,31 марта,ПСБ цифровая лаборатория,Аналитик по данным,Управляющий эксперт по аналитике данных,"Москва, Полный рабочий день",NaN,SQL,https://career.habr.com/vacancies/1000136203
9,30 марта,SM Lab,Аналитик по данным,Аналитик данных (Tableau),"Полный рабочий день, Можно удаленно",NaN,"SQL, Анализ данных, Tableau, DWH",https://career.habr.com/vacancies/1000135388


In [225]:
with pd.ExcelWriter('parsing_results.xlsx') as writer:
    scientists.to_excel(writer, sheet_name='Scientists')
    analytics.to_excel(writer, sheet_name='Analytics')